# Gradient Boosting regression

In [1]:
%matplotlib inline

In [2]:
import math
import pickle

import numpy as np

import matplotlib.pyplot as plt

In [3]:
clf = pickle.load(open('boston_fitted_gbregr.pkl', 'rb'))
feature_names = np.array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT'])
params = clf.get_params()

In [4]:
feature_importance = clf.feature_importances_
if len(feature_names) != feature_importance.shape[0]:
    raise ValueError("The passed feature_names is not valid. The order \
                      of the feature_names should match indices of the \
                      data the model was trained on.")
n_features = feature_importance.shape[0]

# Analysis of Trees

In [5]:
X_train = pickle.load(open('X_train.pkl', 'rb'))  # allow pickle or csv
y_train = pickle.load(open('y_train.pkl', 'rb'))

In [6]:
first_estimator = clf.estimators_.ravel()[0]

In [7]:
first_estimator

DecisionTreeRegressor(criterion='friedman_mse', max_depth=4,
           max_features=None, max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort='auto',
           random_state=<mtrand.RandomState object at 0x7f574206a690>,
           splitter='best')

In [8]:
from illumine.woodland import breakdown_tree

ImportError: No module named 'illumine'